# Tuning Neural Networks with Normalization - Lab

## Introduction

For this lab on initialization and optimization, you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives
You will be able to:
* Build a neural network using Keras
* Normalize your data to assist algorithm convergence
* Implement and observe the impact of various initialization techniques

In [76]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

## Loading the data

The data we'll be working with is data related to facebook posts published during the year of 2014 on the facebook page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set and delete any rows with missing data. Afterwards, briefly preview the data.

In [77]:
#Your code here; load the dataset and drop rows with missing values. Then preview the data.
df = pd.read_csv('dataset_Facebook.csv', header=0, sep=';')
df.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [78]:
df.isna().sum(), df.shape

(Page total likes                                                       0
 Type                                                                   0
 Category                                                               0
 Post Month                                                             0
 Post Weekday                                                           0
 Post Hour                                                              0
 Paid                                                                   1
 Lifetime Post Total Reach                                              0
 Lifetime Post Total Impressions                                        0
 Lifetime Engaged Users                                                 0
 Lifetime Post Consumers                                                0
 Lifetime Post Consumptions                                             0
 Lifetime Post Impressions by people who have liked your Page           0
 Lifetime Post reach by people who lik

In [79]:
df.dropna(inplace=True)

In [80]:
df.isna().sum(), df.shape

(Page total likes                                                       0
 Type                                                                   0
 Category                                                               0
 Post Month                                                             0
 Post Weekday                                                           0
 Post Hour                                                              0
 Paid                                                                   0
 Lifetime Post Total Reach                                              0
 Lifetime Post Total Impressions                                        0
 Lifetime Engaged Users                                                 0
 Lifetime Post Consumers                                                0
 Lifetime Post Consumptions                                             0
 Lifetime Post Impressions by people who have liked your Page           0
 Lifetime Post reach by people who lik

## Defining the Problem

Define X and Y and perform a train-validation-test split.

X will be:
* Page total likes
* Post Month
* Post Weekday
* Post Hour
* Paid
along with dummy variables for:
* Type
* Category

Y will be the `like` column.

In [81]:
#Your code here; define the problem.
y = df.like
y

0        79.0
1       130.0
2        66.0
3      1572.0
4       325.0
5       152.0
6       249.0
7       325.0
8       161.0
9       113.0
10      233.0
11       88.0
12       90.0
13      137.0
14      577.0
15       86.0
16       40.0
17      678.0
18       54.0
19       34.0
20       66.0
21        0.0
22       16.0
23       72.0
24       99.0
25       88.0
26      412.0
27      100.0
28      523.0
29      143.0
        ...  
469     193.0
470     114.0
471     160.0
472      46.0
473     136.0
474      73.0
475      65.0
476     579.0
477     101.0
478      74.0
479      84.0
480     360.0
481       5.0
482     187.0
483      69.0
484      82.0
485      12.0
486      56.0
487      44.0
488     277.0
489      74.0
490      79.0
491     105.0
492     128.0
493     185.0
494     125.0
495      53.0
496      53.0
497      93.0
498      91.0
Name: like, Length: 495, dtype: float64

In [119]:
X = df[[
'Page total likes',
'Post Month',
'Post Weekday',
'Post Hour',
'Paid',
'Type',
'Category']]

X = pd.get_dummies(X, columns=['Type', 'Category'])
X.shape

(495, 12)

In [120]:
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=.2, random_state=123)

## Building a Baseline Model

Next, build a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model.

In [84]:
#Simply run this code block, later you'll modify this model to tune the performance
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

### Evaluating the Baseline

Evaluate the baseline model for the training and validation sets.

In [85]:
#Your code here; evaluate the model with MSE
model.evaluate(X_train, Y_train)
model.evaluate(X_val, Y_val)

99/99 [==============================] - 0s 44us/step


[nan, nan]

In [86]:
#Your code here; inspect the loss function through the history object
hist.history['loss'][:10]

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

> Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.

## Normalize the Input Data

Normalize the input features by subtracting each feature mean and dividing by the standard deviation in order to transform each into a standard normal distribution. Then recreate the train-validate-test sets with the transformed input data.

In [144]:
## standardize/categorize
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X.iloc[:,:5].astype('float')))
X_scaled = X_scaled.join(X.iloc[:,5:])
X_scaled.columns = X.columns
X_scaled.head()

,Page total likes,Post Month,Post Weekday,Post Hour,Paid,Type_Link,Type_Photo,Type_Status,Type_Video,Category_1,Category_2,Category_3
0,1.00496,1.506154,-0.065724,-1.105878,-0.62486,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.00496,1.506154,-0.558655,0.492065,-0.62486,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1.00496,1.506154,-0.558655,-1.105878,-0.62486,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.00496,1.506154,-1.051585,0.492065,1.60036,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.00496,1.506154,-1.051585,-1.105878,-0.62486,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [145]:
X_scaled.shape, X.shape

((495, 12), (495, 12))

In [146]:
X_train, X_val, Y_train, Y_val = train_test_split(X_scaled, y, test_size=.2, random_state=123)

## Refit the Model and Reevaluate

Great! Now refit the model and once again assess it's performance on the training and validation sets.

In [147]:
#Your code here; refit a model as shown above
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [148]:
#Rexamine the loss function
model.evaluate(X_train, Y_train)
model.evaluate(X_val, Y_val)

99/99 [==============================] - 0s 38us/step


[nan, nan]

> Note that you still haven't achieved convergence! From here, it's time to normalize the output data.

## Normalizing the output

Normalize Y as you did X by subtracting the mean and dividing by the standard deviation. Then, resplit the data into training and validation sets as we demonstrated above, and retrain a new model using your normalized X and Y data.

In [177]:
# y_array = y
# y_scale = scaler.fit_transform(y.astype('float'))
# scaler.fit_transform(y_array)
# y_array

Y = df.iloc[:, -1].values

sc_Y = StandardScaler()
Y = sc_Y.fit_transform(Y.reshape(-1, 1))
Y

/opt/conda/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-2.98898902e-01],
       [-1.31048418e-01],
       [-3.51352178e-01],
       [ 4.09930831e+00],
       [ 4.69541596e-01],
       [-7.33498138e-02],
       [ 1.70557921e-01],
       [ 3.27917750e-01],
       [-5.76138309e-02],
       [-1.88747022e-01],
       [ 9.97459979e-02],
       [-2.83162919e-01],
       [-2.88408247e-01],
       [-1.62520383e-01],
       [ 1.00981034e+00],
       [-2.77917591e-01],
       [-4.19541437e-01],
       [ 1.30879402e+00],
       [-3.64465497e-01],
       [-4.51013403e-01],
       [-3.48729514e-01],
       [-5.61165283e-01],
       [-5.13957335e-01],
       [-3.09389557e-01],
       [-2.43822962e-01],
       [-2.83162919e-01],
       [ 7.34430641e-01],
       [-2.54313617e-01],
       [ 1.07013161e+00],
       [-1.04821780e-01],
       [-1.36293745e-01],
       [-4.58881394e-01],
       [-2.61418652e-02],
       [-2.33332306e-01],
       [-3.64465497e-01],
       [-3.92551842e-02],
       [-2.64804272e-01],
       [-2.98898902e-01],
       [ 1.4

In [161]:
#Your code here: redefine Y after normalizing the data.
scaler = StandardScaler()

# Y_scaled = (df["like"]-np.mean(df["like"]))/(np.std(df["like"]))
# Y_scaled

0     -0.309011
1     -0.151644
2     -0.349124
3      4.297815
4      0.450051
5     -0.083760
6      0.215544
7      0.450051
8     -0.055990
9     -0.204100
10     0.166174
11    -0.281240
12    -0.275069
13    -0.130045
14     1.227627
15    -0.287411
16    -0.429350
17     1.539274
18    -0.386151
19    -0.447863
20    -0.349124
21    -0.552774
22    -0.503404
23    -0.330610
24    -0.247298
25    -0.281240
26     0.718500
27    -0.244213
28     1.061003
29    -0.111531
         ...   
469    0.042750
470   -0.201014
471   -0.059076
472   -0.410836
473   -0.133130
474   -0.327524
475   -0.352209
476    1.233798
477   -0.241127
478   -0.324439
479   -0.293582
480    0.558048
481   -0.537346
482    0.024236
483   -0.339867
484   -0.299754
485   -0.515747
486   -0.379980
487   -0.417007
488    0.301942
489   -0.324439
490   -0.309011
491   -0.228784
492   -0.157815
493    0.018065
494   -0.167072
495   -0.389237
496   -0.389237
497   -0.265812
498   -0.271983
Name: like, Length: 495,

In [ ]:
#Your code here; create training and validation sets as before. Use random seed 123.

In [ ]:
#Your code here; rebuild a simple model using a relu layer followed by a linear layer. (See our code snippet above!)

Again, reevaluate the updated model.

In [ ]:
#Your code here; MSE

In [ ]:
#Your code here; loss function

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.)

## Using Weight Initializers

Below, take a look at the code provided to see how to modify the neural network to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe is the strongest.

##  He Initialization

In [19]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, kernel_initializer= "he_normal",
                activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val),verbose=0)

In [20]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [21]:
print(MSE_train)
print(MSE_val)

1.0392949820359312
0.8658544030836142


## Lecun Initialization

In [22]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, 
                kernel_initializer= "lecun_normal", activation='tanh'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [23]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [24]:
print(MSE_train)
print(MSE_val)

1.0307351124941144
0.9292005788570431


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop

In [25]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "rmsprop" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [26]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [27]:
print(MSE_train)
print(MSE_val)

1.020200641136699
0.9421919606123765


## Adam

In [28]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [29]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [30]:
print(MSE_train)
print(MSE_val)

1.0219766410555322
0.9477664629838952


## Learning Rate Decay with Momentum


In [31]:
np.random.seed(123)
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= sgd ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [32]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [33]:
print(MSE_train)
print(MSE_val)

0.8667952792265361
1.1040802536956849


## Selecting a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [ ]:
#Your code here

## Summary  

In this lab, you worked to ensure your model converged properly. Additionally, you also investigated the impact of varying initialization and optimization routines.